# Focused Ion Beam Molecular Dynamics (fibmd) Tool

In [ ]:
from IPython.display import display, clear_output, Javascript
import ipywidgets as widgets
import numpy as np
from string import Template
import hublib.use
import hublib
import py3Dmol
from IPython.display import FileLink, FileLinks
from hublib.ui import Submit

In [ ]:
%use openmpi-1.6.3-gnu-4.7.2

# Creating widgets for input

In [ ]:
def beamcurrent(dtin):
    Beam_current = 1.602e-19/(dtin*1e-12)*1e9
    return Beam_current
def surfflux(dtin,fin):
    surf_flux = 1/dtin/((fin/2.355*4*1e-7)**2*np.pi/4)
    return surf_flux
def estimatedsize(xdimin,silat,ydimin,zdimin,ain,dtin,nin):
    estimated_size=6.5e-05*(np.floor(xdimin.value/silat) * 
                      np.floor(ydimin.value/silat) * 
                      np.floor(zdimin.value/silat))*8*(.1/.02e-3 /(ain/10) + (dtin-.1)/.2e-3/(ain))*nin
    return estimated_size

In [ ]:
def uncertainty_quantification1 (dtin,dtin_std):
   
    dtin_n = np.random.normal(dtin.value, dtin_std.value)

    return beamcurrent(dtin_n)

def uncertainty_quantification2 (dtin,dtin_std):
   
    dtin_n = np.random.normal(dtin.value, dtin_std.value)
    fin_n = np.random.normal(fin.value, fin_std.value)
    return surfflux(dtin_n,fin_n)


def uncertainty_quantification3 (xdimin,silat,ydimin,zdimin,ain,dtin,nin,fin,ain_std,dtin_std,nin_std,fin_std):
   
  
    ain_n = np.random.normal(ain.value, ain_std.value)
    dtin_n = np.random.normal(dtin.value, dtin_std.value)
    nin_n = np.random.normal(nin.value, nin_std.value)
    fin_n = np.random.normal(fin.value, fin_std.value)
    
    
    
    return estimatedsize(xdimin,silat,ydimin,zdimin,ain_n,dtin_n,nin_n)


def count_range_in_list(list_count, lower_bound, upper_bound):
    list_count.sort(reverse=False);
    freq_list_count = [0]*101;
    width = (upper_bound-lower_bound)/100;
    i=0;
    for x in list_count:
        if lower_bound+i*width<=x<lower_bound+(i+1)*width:
            freq_list_count[i]+=1
        else:
            i+=1
    return freq_list_count
def count_range_in_list1(list_count, lower_bound, upper_bound):
    list_count.sort(reverse=False);
    freq_list_count = [0]*100001;
    width = (upper_bound-lower_bound)/100000;
    i=0;
    for x in list_count:
        if lower_bound+i*width<=x<lower_bound+(i+1)*width:
            freq_list_count[i]+=1
        else:
            i+=1
    return freq_list_count

In [ ]:
#title = widgets.Label(value="FIBMD Tool", width=10)

box_layout = widgets.Layout(display='flex',
                            justify_content='center')
text_layout = widgets.Layout(width = '175px')
text_layout1 = widgets.Layout(width = '50px')
text_layout2 = widgets.Layout(width ='90px',
                              position = 'right')

#ion energy input
keVin = widgets.BoundedFloatText(
    value=2000,
    min=500,
    max=60000,
    step=100,
    layout=text_layout2,
    tooltip = 'Beam energy, in keV'
)
keVin_std = widgets.BoundedFloatText(
    value=200,
    min=0,
    max=400,
    step=20,
    layout=text_layout2,
    tooltip = 'Beam energy, in keV'
)
#x dimension for simulation box
xdimin = widgets.BoundedFloatText(
    value=5,
    min=4,
    max=40,
    step=1,
    layout=text_layout1
)
#y dimension for simulation box
ydimin = widgets.BoundedFloatText(
    value=5,
    min=4,
    max=40,
    step=1,
    layout=text_layout1
)
#z dimension for simulation box
zdimin = widgets.BoundedFloatText(
    value=2,
    min=1,
    max=40,
    step=1,
    layout=text_layout1
)
# number of processors to use in MPI
pin = widgets.BoundedIntText(
    value = 2,
    min = 1,
    max = 2,
    step = 1,
    layout=text_layout2
)
pin_std = widgets.BoundedIntText(
    value = 0.1,
    min = 1,
    max = 2,
    step = 0.1,
    layout=text_layout2
)
#temperature setting for the silicon target
tempin = widgets.BoundedFloatText(
    value = 300.0,
    step = 10.0,
    layout = text_layout2,
    min = 10.0,
    max = 2000.
)
tempin_std = widgets.BoundedFloatText(
    value = 30.0,
    step = 5.0,
    layout = text_layout2,
    min = 0.0,
    max = 60.
)
#mumber of gallium ions to fire
nin = widgets.BoundedIntText(
    value = 1,
    min = 1,
    max = 100,
    step = 1,
    layout=text_layout2
)

nin_std = widgets.BoundedFloatText(
    value = 0.1,
    step = 0.1,
    layout=text_layout2,
    min = 0.1
)
#output frequency of full .xyz file, in timesteps
ain  = widgets.BoundedIntText(
    value = 500,
    min = 0,
    max = 1e6,
    step = 100,
    layout=text_layout2
)
ain_std  = widgets.BoundedIntText(
    value = 50,
    min = 0,
    max = 1e6,
    step = 10,
    layout=text_layout2
)
#time between ion impacts
dtin = widgets.BoundedFloatText(
    value = 1,
    step = 0.1,
    layout=text_layout2,
    min = 0.1
)
dtin_std = widgets.BoundedFloatText(
    value = 0.1,
    step = 0.1,
    layout=text_layout2,
    min = 0.1
)
#angle of incidence, from z axis
phizin = widgets.BoundedFloatText(
    value = 0.0,
    step = 1,
    min = 0,
    max = 89,
    layout=text_layout2
)
phizin_std = widgets.BoundedFloatText(
    value = 5.0,
    step = 1,
    min = 0,
    max = 10,
    layout=text_layout2
)
#angle of incidence, from x axis
phixyin = widgets.FloatText(
    value = 0.0,
    step = 1,
    layout=text_layout2
)
phixyin_std = widgets.FloatText(
    value = 5.0,
    step = 1,
    layout=text_layout2
)
#FWHM of incident beam distribution, in nm
fin = widgets.BoundedFloatText(
    value = 2,
    step = 0.1,
    layout=text_layout2,
    min = 0.1
)
fin_std = widgets.BoundedFloatText(
    value = 0.2,
    step = 0.1,
    layout=text_layout2,
    min = 0.1
)

dimlabel = widgets.Label(" Target dimensions, nm",layout=box_layout)

xlabel = widgets.Label("    X:", position='right')
ylabel = widgets.Label("    Y:", position='right')
zlabel = widgets.Label("    Z:", position='right')
xbox = widgets.HBox([xlabel,xdimin],layout = text_layout)
ybox = widgets.HBox([ylabel,ydimin],layout = text_layout)
zbox = widgets.HBox([zlabel,zdimin],layout = text_layout)

keVlabel = widgets.Label("Ion Energy (eV) :")
keVlabel_std = widgets.Label("Standard Deviation of Ion Energy (eV) ")
proclabel = widgets.Label("# of Processors:")
templabel = widgets.Label("Temperature (K):")
templabel_std = widgets.Label("Standard Deviation of Temperature (K):")
ionlabel = widgets.Label("Mean # of Ions to fire:")
ionlabel_std = widgets.Label("Standard Deviation # of Ions to fire:")
outlabel = widgets.Label("Mean of Output frequency:")
outlabel_std = widgets.Label("Standard Deviation of Output frequency:")
dtlabel = widgets.Label("Mean of Δ t$_{ion}$ (ps)")
dtlabel_std = widgets.Label("Stabdard Deviation ofΔ t$_{ion}$ (ps)")
anglezlabel = widgets.Label("Beam Angle θ:")
anglezlabel_std = widgets.Label("Standard Deviation of Beam Angle θ:")
anglexylabel = widgets.Label("Beam Angle Φ:")
anglexylabel_std = widgets.Label("Standard Deviation of Beam Angle Φ:")
fwhmlabel = widgets.Label("Mean of FWHM of beam (nm):")
fwhmlabel_std = widgets.Label("Standard Deviaton of FWHM of beam (nm):")
beamlabel = widgets.Label("Mean Effective Beam Current (nA): ")
beamlabel_std = widgets.Label("Standard Deviation of Effective Beam Current (nA): ")
sizelabel = widgets.Label("Mean Estimated Total File Size (MB)")
sizelabel_std = widgets.Label("Standard Deviation of Estimated Total File Size (MB)")
fluxlabel = widgets.Label("Mean 4σ Surface Flux (ions/cm$^2$ps): ")
fluxlabel_std = widgets.Label("Standard Deviation of 4σ Surface Flux (ions/cm$^2$ps): ")

dimin = widgets.HBox([xbox,ybox,zbox])
vlabels = widgets.VBox([keVlabel,proclabel,templabel, ionlabel, ionlabel_std,
                        outlabel,outlabel_std, dtlabel,dtlabel_std, anglezlabel, 
                        anglexylabel, fwhmlabel,fwhmlabel_std])
vinput = widgets.VBox([keVin,pin,tempin, nin,nin_std, ain,ain_std, dtin,dtin_std,
                       phizin, phixyin, fin,fin_std])

inputs2 = widgets.HBox([vlabels,vinput])

#silicon lattice constant, nm
silat = 0.5431

## dynamically updating boxes for current, estimated file size, flux
BeamCurrent = widgets.Text(
    value = '%.2f'%(0.),
    layout = text_layout2,
    position = 'right'
)

SurfFlux = widgets.Text(
   value = '%.2f'%(0.),
    layout = text_layout2,
        position = 'right'
)

    #6.44e-5 is estimate of MB per atom per snapshot
    #floors of dimin are unit cells, 8 is atoms per unit cell
    #.1/.02e-3 is estimating number of timesteps in fast regime (lasting .1 ps, with .02e-3 step size)
    #(dtin.value-.1)/.2e-3 is # of timesteps during the rest of the time
    #ain.value/10 and ain.value normalize total timesteps by output frequency in each case
    #whole process happens nin.value times
EstimatedSize = widgets.Text( 
    value = '%.2f'%(0.),
        layout = text_layout2,
        position = 'right'
)
BeamCurrent_std = widgets.Text(
    value = '%.2f'%(0.),
    layout = text_layout2,
    position = 'right'
)

SurfFlux_std = widgets.Text(
   value = '%.2f'%(0.),
    layout = text_layout2,
        position = 'right'
)

    #6.44e-5 is estimate of MB per atom per snapshot
    #floors of dimin are unit cells, 8 is atoms per unit cell
    #.1/.02e-3 is estimating number of timesteps in fast regime (lasting .1 ps, with .02e-3 step size)
    #(dtin.value-.1)/.2e-3 is # of timesteps during the rest of the time
    #ain.value/10 and ain.value normalize total timesteps by output frequency in each case
    #whole process happens nin.value times
EstimatedSize_std = widgets.Text( 
    value = '%.2f'%(0.),
        layout = text_layout2,
        position = 'right'
)
# vlabels2 = widgets.VBox([beamlabel,fluxlabel,sizelabel])
# outlabels = widgets.VBox([BeamCurrent,SurfFlux,EstimatedSize])
# outlabels2 = widgets.HBox([vlabels2,outlabels])
# input1 = widgets.VBox([dimlabel,dimin,inputs2],layout=widgets.Layout(border='solid'))
# FullBox = widgets.VBox([input1,outlabels2])



In [ ]:
from __future__ import print_function
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
from collections import OrderedDict
from numpy import *
from string import Template
#import hublib.use
import hublib
from IPython.display import FileLink, FileLinks, clear_output
from hublib.ui import RunAllButton
import py3Dmol
from mpl_toolkits import mplot3d

# tpl imports
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot
import numpy as np

plotly.offline.init_notebook_mode(connected=True)

def plotly_plot(out):

    BeamCurrent1= [0]*10000;
    SurfFlux1 = [0]*10000;
    EstimatedSize1 = [0]*10000;
    for i in range (10000):
        BeamCurrent1[i]= uncertainty_quantification1(dtin,dtin_std)
        SurfFlux1[i]=uncertainty_quantification2 (dtin,dtin_std)
        EstimatedSize1[i]=uncertainty_quantification3 (xdimin,silat,ydimin,zdimin,ain,dtin,nin,fin,ain_std,dtin_std,nin_std,fin_std)
    BeamCurrent_mean = mean(BeamCurrent1)
    SurfFlux_mean = mean(SurfFlux1)
    EstimatedSize_mean = mean(EstimatedSize1)
    BeamCurrent_min = min(BeamCurrent1)
    SurfFlux_min = min(SurfFlux1)
    EstimatedSize_min = min(EstimatedSize1)
    BeamCurrent_max = max(BeamCurrent1)
    SurfFlux_max = max(SurfFlux1)
    EstimatedSize_max = max(EstimatedSize1)   
    
    freq_BeamCurrent= count_range_in_list(BeamCurrent1,min(BeamCurrent1), max(BeamCurrent1));
    freq_SurfFlux = count_range_in_list(SurfFlux1, min(SurfFlux1), max(SurfFlux1));
    freq_EstimatedSize = count_range_in_list(EstimatedSize1, min(EstimatedSize1), max(EstimatedSize1));

 
    # transfer calculated dimenstions to output widgets
#     op1.value = "%0.03f" % l
#     op2.value = "%0.03f" % d
#     op3.value = "%0.03f" % l_min
#     op4.value = "%0.03f" % d_min
#     op5.value = "%0.03f" % l_max
#     op6.value = "%0.03f" % d_max
    
       ## dynamically updating boxes for current, estimated file size, flux
    BeamCurrent.value ='%0.3f'%(mean(BeamCurrent1))
    BeamCurrent_std.value ='%0.3f'%(std(BeamCurrent1))

    SurfFlux.value = '%1.4E'%(mean(SurfFlux1))
    SurfFlux_std.value = '%1.4E'%(std(SurfFlux1))
        #6.44e-5 is estimate of MB per atom per snapshot
        #floors of dimin are unit cells, 8 is atoms per unit cell
        #.1/.02e-3 is estimating number of timesteps in fast regime (lasting .1 ps, with .02e-3 step size)
        #(dtin.value-.1)/.2e-3 is # of timesteps during the rest of the time
        #ain.value/10 and ain.value normalize total timesteps by output frequency in each case
        #whole process happens nin.value times
    EstimatedSize.value = '%0.3f'%(mean(EstimatedSize1))
    EstimatedSize_std.value = '%0.3f'%(std(EstimatedSize1))
#     vlabels2 = widgets.VBox([beamlabel,fluxlabel,sizelabel])
#     outlabels = widgets.VBox([BeamCurrent,SurfFlux,EstimatedSize])
#     outlabels2 = widgets.HBox([vlabels2,outlabels])
#     input1 = widgets.VBox([dimlabel,dimin,inputs2],layout=widgets.Layout(border='solid'))
#     FullBox = widgets.VBox([input1,outlabels2])
    
    trace4 = go.Bar(
                x = list(np.arange(BeamCurrent_min,BeamCurrent_max,(BeamCurrent_max-BeamCurrent_min)/100)), 
                y = freq_BeamCurrent
                
    )
    
    trace5 = go.Bar(
                x = list(np.arange(SurfFlux_min,SurfFlux_max,(SurfFlux_max-SurfFlux_min)/100)),
                y = freq_SurfFlux 
    )
    trace6 = go.Bar(
                x = list(np.arange(EstimatedSize_min,EstimatedSize_max,(EstimatedSize_max-EstimatedSize_min)/100)),
                y = freq_EstimatedSize
    )
    
    fig4 = go.Figure(data=[trace4])
    layout4 = go.Layout(
        title='Distribution of Effective Beam Current (nA)',
        xaxis=dict(title='Effective Beam Current (nA)'),
        yaxis=dict(title='Number of Occurances')
    )
    fig4.layout.update(layout4)

    fig5 = go.Figure(data=[trace5])
    layout5 = go.Layout(
        title='Distribution of 4σ Surface Flux',
        xaxis=dict(title='4σ Surface Flux '),
        yaxis=dict(title='Number of Occurances')
    )
    fig5.layout.update(layout5)

    fig6 = go.Figure(data=[trace6])
    layout6 = go.Layout(
        title='Distribution of Estimated Total File Size (MB)',
        xaxis=dict(title='Estimated Total File Size (MB)'),
        yaxis=dict(title='Number of Occurances')
    )
    fig6.layout.update(layout6)

#     fig4 = go.Figure(data=[trace4])
#     fig4.update_layout(title_text='Distribution of Effective Beam Current (nA)', title_x=0.5,
#                       xaxis_title="Effective Beam Current (nA)",
#                       yaxis_title="Number of Occurances"
#                       )
    
#     fig5 = go.Figure(data=[trace5])
#     fig5.update_layout(title_text='Distribution of 4σ Surface Flux', title_x=0.5,
#                       xaxis_title="4σ Surface Flux ",
#                       yaxis_title="Number of Occurances"
#                       )
#     fig6 = go.Figure(data=[trace6])
#     fig6.update_layout(title_text='Distribution of Estimated Total File Size (MB)', title_x=0.5,
#                       xaxis_title="Estimated Total File Size (MB)",
#                       yaxis_title="Number of Occurances"
#                       )

    with out:
        clear_output()
        iplot(fig4)
        iplot(fig5)
        #iplot(fig6)

In [ ]:
#more detailed descriptions of input parameters for tab view
topic_list = ['Dimensions','Ion Energy','Processors','Temperature','Ions','Output','Δt ion','θ','Φ','FWHM','Current','Flux','Size']
topic_descriptions = ['Simulation domain dimensions in nm. In this program, this input is rounded down to the nearest multiple of silicon\'s lattice constant (0.5431 nm).',
                     'Kinetic Energy of the Ga ion being fired, in eV',
                     'The number of processors to use, but limited to 2 for local NanoHUB simulations. Using 2 should always be preferable, but very small domains may require 1 due to the way parallel calculations are implemented.',
                     'Silicon thermostatted temperature, applied to a 1.5 nm thick outer boundary in the x-y plane. This is implemented as a Berendsen thermostat with τ = 0.1 (ps).',
                     'Total number of ions to fire at the surface in series',
                     """This determines the xyz file snapshot frequency as every 'output frequency' timesteps during large steps (0.2 fs) and 'output frequency'/10 steps during small steps (0.02 fs). Small vs. big timesteps are determined by ion energies, as described in the abstract.""",
                     'Time between each ion impact, in ps. Total simulation runtime is (Δt ion)x(Ions).',
                     'Off-normal angle of incidence for ion beam.',
                     'Projected x-y plane beam direction from x axis, only relevant for θ ≠ 0.',
                     'Full width half maximum of the Gaussian ion distribution, centered in the x-y plane.',
                     'Beam current conversion, based on time between impacts.',
                     'Beam flux conversion, based on time between impacts and FWHM. This is calculated for an area corresponding to 4 standard deviations (σ) of the random ion distribution.',
                     'Estimated final .xyz file size; it will tend to overestimate very small/low energy cases and may underestimate large and high energy cases. This file is suggested for analysis with Ovito and the provided OvitoTemplate.ovito file.']

topic_dict = dict(zip(topic_list,topic_descriptions))

desc_layout1 = widgets.Layout(width = '280px')
desc_layout2 = widgets.Layout(width = '280px',
                             height = '400px'
)

#topic menu for descriptions
topic_widg = widgets.Dropdown(
    options=topic_list,
    description='Topic:',
    layout = desc_layout1,
)

#descriptions for each topic
topicdesc_widg = widgets.Textarea(
    value=topic_dict[topic_widg.value],
    layout = desc_layout2,
)

descriptions = widgets.VBox([topic_widg,topicdesc_widg])

plot_out2 = widgets.Output(layout={'width': '75%', 'border': '1px solid green'})


def run_button_clicked3(b):
    b.button_style='warning'
    plotly_plot(plot_out2)
    b.button_style=''
button3 = widgets.Button(description="SIMULATE")
button3.on_click(run_button_clicked3)
left = widgets.VBox([ button3], layout={'min_width': '380px'})
widgets.HBox([left,plot_out2], layout= {'justify_content': 'space-between'})


vlabels2 = widgets.VBox([beamlabel,beamlabel_std,fluxlabel,fluxlabel_std,sizelabel,sizelabel_std])
outlabels = widgets.VBox([BeamCurrent,BeamCurrent_std,SurfFlux,SurfFlux_std,EstimatedSize,EstimatedSize_std])
outlabels2 = widgets.HBox([vlabels2,outlabels])
input1 = widgets.VBox([dimlabel,dimin,inputs2],layout=widgets.Layout(border='solid'))
FullBox = widgets.VBox([input1,outlabels2,button3])
all_in=widgets.HBox([FullBox,plot_out2], layout= {'justify_content': 'space-between'})

In [ ]:
tab_view = widgets.Tab()
tab_view.children = [all_in,descriptions]
tab_view.set_title(0, 'Input Parameters')
tab_view.set_title(1, 'Input Descriptions')
display(tab_view)

In [ ]:
!touch out.log data/mdrun2.xyz data/3dmol2.xyz
#file download links
outfile = FileLink("out.log")
xyzfile = FileLink("data/mdrun2.xyz")
ovitofile = FileLink("data/OvitoTemplate.ovito")
display(outfile,xyzfile,ovitofile)

In [ ]:
##For determining how to distribute processor load in domain
#list of prime factors for some number 'n'
def prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors

#determine 3 integer factors of nc that are closest together
def primefactors3(nc):
    a=prime_factors(nc)
    z = np.ones(3,dtype=np.int8)
    zb = []

    if len(a) < 3:
        z[3-len(a):] = a
        b = range(0)
    else:
        z = z = a[len(a)-3:]
        b = range(len(a)-4,-1,-1)

    for i in b:
        indmin = np.argmin(z)
        z[indmin]=z[indmin]*a[i]    

    zb.append(z[np.argmax(z)])
    zb.append(int(np.sum(z)-z[np.argmin(z)]-z[np.argmax(z)]))
    zb.append(z[np.argmin(z)])
    return zb

#Create the input file from siga.in.template
def get_template():
    procs = primefactors3(pin.value)
    if ain.value == 0:
        atmoutin1 = -1
        atmoutin2 = -1
    else:
        atmoutin1 = int(ain.value/10)
        atmoutin2 = ain.value
        dims = [int(xdimin.value/silat), int(ydimin.value/silat), int(zdimin.value/silat)]
        
    inputs = list([str(keVin.value),str(dims[0]),str(dims[1]),str(dims[2]),
                         str(procs[0]),str(procs[1]),str(procs[2]),str(tempin.value),str(nin.value),str(atmoutin1),str(atmoutin2),
                         str(dtin.value*1e-12),str(fin.value),str(phizin.value),str(phixyin.value)])
    
    tags = list(['keV','xdim','ydim','zdim',
                 'procsx','procsy','procsz','Ttar1','nlj','atmout1','atmout2',
                 'dtion','fwhm','phiz','phixy'])

    input_dict = dict(zip(tags, inputs))
    temp_contents = open('siga.in.template').read()
    tempstr = Template(temp_contents)
    return tempstr.substitute(input_dict)


##Callback functions 
def BC_cb(change):
    BeamCurrent.value = str(1.602e-19/(dtin.value*1e-12)*1e9) #atoms/ps to nA
    
def ES_cb(change):
    if ain.value <= 0:
        EstimatedSize.value = '0'
    else:
        EstimatedSize.value = '%0.3f'%(6.5e-05*(np.floor(xdimin.value/silat) * 
                      np.floor(ydimin.value/silat) * 
                      np.floor(zdimin.value/silat))*8*
             (.1/.02e-3 /(ain.value/10) + (dtin.value-.1)/.2e-3/(ain.value))*nin.value)

def SF_cb(change):
    SurfFlux.value = '%1.4E'%(1/dtin.value/((fin.value/2.355*4*1e-7)**2*np.pi/4))
        
def topic_cb(change):
    topicdesc_widg.value = topic_dict[topic_widg.value]
    
#button click run function
def my_start(s):
    rname = s.make_rname(pin.value)
    !rm siga.in
    with open('siga.in', 'w') as tfile:
        tfile.write(get_template())

    # run locally for now, also split output to out.log and stdout
    submit_str = '--local mpirun -np %i bin/mdrun2 | tee out.log'%(pin.value)
    s.run(rname, submit_str)
    
#button click for updating the visualization window
def updateviewer(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))
UV = widgets.Button(button_style='info',description="Update Viewer")
UV.on_click(updateviewer)



## Observers, for dynamic input/output box updates
dtin.observe(BC_cb,names = 'value')
dtin.observe(ES_cb,names = 'value')
dtin.observe(SF_cb,names = 'value')
ain.observe(ES_cb,names = 'value')
nin.observe(ES_cb,names = 'value')
xdimin.observe(ES_cb,names = 'value')
ydimin.observe(ES_cb,names = 'value')
zdimin.observe(ES_cb,names = 'value')
fin.observe(SF_cb,names = 'value')
topic_widg.observe(topic_cb, names='value')

#definition for the visualization window
p = py3Dmol.view(width=520,height=520)

In [ ]:
test = Submit(start_func=my_start, cachename='SubmitTest')
test

In [ ]:
display(UV)

In [ ]:
#3dmol viewer
#display(widgets.Text(value = 'control+click to slide'))
#print('\n')
file = 'data/3dmol2.xyz'
xyzout = open(file,'r').read()
p.clear()


p.addModel(xyzout,'xyz')
p.setStyle({'sphere':{}})
p.setBackgroundColor('0xeeeeee')
#p.clear()
#p.zoom()
p.render()
#display(widgets.Text(value = '\n'))